In [1]:
def testLPT():
    import vtk
    seeds = vtk.vtkPointSource()
    seeds.SetNumberOfPoints(10)
    seeds.SetRadius(4)
    seeds.Update()
    seedPD = seeds.GetOutput()
    seedData = seedPD.GetPointData()

    partVel = vtk.vtkDoubleArray()
    partVel.SetNumberOfComponents(3)
    partVel.SetNumberOfTuples(seedPD.GetNumberOfPoints())
    partVel.SetName('InitialVelocity')

    partDens = vtk.vtkDoubleArray()
    partDens.SetNumberOfComponents(1)
    partDens.SetNumberOfTuples(seedPD.GetNumberOfPoints())
    partDens.SetName('ParticleDensity')

    partDiam = vtk.vtkDoubleArray()
    partDiam.SetNumberOfComponents(1)
    partDiam.SetNumberOfTuples(seedPD.GetNumberOfPoints())
    partDiam.SetName('ParticleDiameter')

    partVel.FillComponent(0, 200)
    partVel.FillComponent(1, 5)
    partVel.FillComponent(2, 1)
    partDens.FillComponent(0, 1920)
    partDiam.FillComponent(0, 0.1)

    seedData.AddArray(partVel)
    seedData.AddArray(partDens)
    seedData.AddArray(partDiam)

    # Not sure about the DeepCopy implementation

    wavelet = vtk.vtkRTAnalyticSource()
    wavelet.Update()
    waveletImg = wavelet.GetOutput()
    cd = waveletImg.GetCellData()

    flowVel = vtk.vtkDoubleArray()
    flowVel.SetNumberOfComponents(3)
    flowVel.SetNumberOfTuples(waveletImg.GetNumberOfCells())
    flowVel.SetName('FlowVelocity')

    flowDens = vtk.vtkDoubleArray()
    flowDens.SetNumberOfComponents(1)
    flowDens.SetNumberOfTuples(waveletImg.GetNumberOfCells())
    flowDens.SetName('FlowDensity')

    flowDynVisc = vtk.vtkDoubleArray()
    flowDynVisc.SetNumberOfComponents(1)
    flowDynVisc.SetNumberOfTuples(waveletImg.GetNumberOfCells())
    flowDynVisc.SetName('FlowDynamicViscosity')

    flowVel.FillComponent(0, -0.3)
    flowVel.FillComponent(1, -0.3)
    flowVel.FillComponent(2, -0.3)
    flowDens.FillComponent(0, 1000)
    flowDynVisc.FillComponent(0, 0.894)

    cd.AddArray(flowVel)
    cd.AddArray(flowDens)
    cd.AddArray(flowDynVisc)

    surface = vtk.vtkDataSetSurfaceFilter()
    surface.SetInputConnection(wavelet.GetOutputPort())
    surface.Update()
    surfacePd = surface.GetOutput()

    surfaceTypeTerm = vtk.vtkDoubleArray()
    surfaceTypeTerm.SetNumberOfComponents(1)
    surfaceTypeTerm.SetName('SurfaceType')
    surfaceTypeTerm.SetNumberOfTuples(surfacePd.GetNumberOfCells())
    surfaceTypeTerm.FillComponent(0, vtk.vtkLagrangianBasicIntegrationModel.SURFACE_TYPE_TERM)
    surfacePd.GetCellData().AddArray(surfaceTypeTerm)

    surfacePass = vtk.vtkPlaneSource()
    surfacePass.SetOrigin(-10, -10, 0)
    surfacePass.SetPoint1(10, -10, 0)
    surfacePass.SetPoint2(-10, 10, 0)
    surfacePass.Update()
    passPd = surfacePass.GetOutput()

    surfaceTypePass = vtk.vtkDoubleArray()
    surfaceTypePass.SetNumberOfComponents(1)
    surfaceTypePass.SetName('SurfaceType')
    surfaceTypePass.SetNumberOfTuples(passPd.GetNumberOfCells())
    surfaceTypePass.FillComponent(0, vtk.vtkLagrangianBasicIntegrationModel.SURFACE_TYPE_PASS)
    passPd.GetCellData().AddArray(surfaceTypePass)

    surfaceBounce = vtk.vtkPlaneSource()
    surfaceBounce.SetOrigin(-2, -2, -2)
    surfaceBounce.SetPoint1(5, -2, -2)
    surfaceBounce.SetPoint2(-2, 5, -2)
    surfaceBounce.Update()
    bouncePd = surfaceBounce.GetOutput()

    surfaceTypeBounce = vtk.vtkDoubleArray()
    surfaceTypeBounce.SetNumberOfComponents(1)
    surfaceTypeBounce.SetName('SurfaceType')
    surfaceTypeBounce.SetNumberOfTuples(bouncePd.GetNumberOfCells())
    surfaceTypeBounce.FillComponent(0, vtk.vtkLagrangianBasicIntegrationModel.SURFACE_TYPE_BOUNCE)
    bouncePd.GetCellData().AddArray(surfaceTypeBounce)

    groupSurface = vtk.vtkMultiBlockDataGroupFilter()
#     groupSurface.AddInputDataObject(surfacePd)
    groupSurface.AddInputDataObject(passPd)
    groupSurface.AddInputDataObject(bouncePd)

    groupFlow = vtk.vtkMultiBlockDataGroupFilter()
    groupFlow.AddInputDataObject(waveletImg)

    ugFlow = vtk.vtkImageDataToPointSet()
    ugFlow.AddInputData(waveletImg)

    groupSeed = vtk.vtkMultiBlockDataGroupFilter()
    groupSeed.AddInputDataObject(seedPD)

    integrator = vtk.vtkRungeKutta2()

    integrationModel = vtk.vtkLagrangianMatidaIntegrationModel()
    integrationModel.SetInputArrayToProcess(0, 1, 0, vtk.vtkDataObject.FIELD_ASSOCIATION_POINTS, 'InitialVelocity')
    integrationModel.SetInputArrayToProcess(2, 0, 0, vtk.vtkDataObject.FIELD_ASSOCIATION_CELLS, 'SurfaceType')
    integrationModel.SetInputArrayToProcess(3, 0, 0, vtk.vtkDataObject.FIELD_ASSOCIATION_CELLS, 'FlowVelocity')
    integrationModel.SetInputArrayToProcess(4, 0, 0, vtk.vtkDataObject.FIELD_ASSOCIATION_CELLS, 'FlowDensity')
    integrationModel.SetInputArrayToProcess(5, 0, 0, vtk.vtkDataObject.FIELD_ASSOCIATION_CELLS, 'FlowDynamicViscosity')
    integrationModel.SetInputArrayToProcess(6, 1, 0, vtk.vtkDataObject.FIELD_ASSOCIATION_POINTS, 'ParticleDiameter')
    integrationModel.SetInputArrayToProcess(7, 1, 0, vtk.vtkDataObject.FIELD_ASSOCIATION_POINTS, 'ParticleDensity')
    integrationModel.SetNumberOfTrackedUserData(13)

    import sys
    tracker = vtk.vtkLagrangianParticleTracker()
    tracker.SetIntegrator(None)
    tracker.SetIntegrationModel(None)
    print(tracker)
    if (tracker.GetSource() != None or tracker.GetSurface() != None):
        print('Incorrect Input Initialization', file=sys.stderr)
    #     return
    tracker.SetIntegrator(integrator)
    if (tracker.GetIntegrator() != integrator):
        print('Incorrect Integrator', file=sys.stderr)
        return
    tracker.SetIntegrationModel(integrationModel)
    if (tracker.GetIntegrationModel() != integrationModel):
        print('Incorrect Integration Model', file=sys.stderr)
        return

    tracker.SetInputConnection(groupFlow.GetOutputPort())
    tracker.SetStepFactor(0.1)
    tracker.SetStepFactorMin(0.1)
    tracker.SetStepFactorMax(0.1)
    tracker.SetMaximumNumberOfSteps(150)
    tracker.SetSourceConnection(groupSeed.GetOutputPort())
    tracker.SetSurfaceConnection(groupSurface.GetOutputPort())
#     tracker.SetSurfaceData(surfacePd)
    tracker.SetCellLengthComputationMode(vtk.vtkLagrangianParticleTracker.STEP_CUR_CELL_VEL_DIR)
    tracker.AdaptiveStepReintegrationOn()
    tracker.GenerateParticlePathsOutputOff()
    tracker.Update()
    tracker.GenerateParticlePathsOutputOn()
#     tracker.SetInputConnection(ugFlow.GetOutputPort())
#     tracker.SetMaximumNumberOfSteps(30)
#     tracker.SetCellLengthComputationMode(vtk.vtkLagrangianParticleTracker.STEP_CUR_CELL_DIV_THEO)
    tracker.Update()
    tracker.SetMaximumNumberOfSteps(-1)
    tracker.SetMaximumIntegrationTime(10.0)
    tracker.Update()
#     tracker.SetInputData(waveletImg)
#     tracker.SetSourceData(seedPD)
#     tracker.SetMaximumNumberOfSteps(300)
#     tracker.SetMaximumIntegrationTime(-1.0)
#     tracker.SetSurfaceConnection(groupSurface.GetOutputPort())
#     tracker.SetCellLengthComputationMode(vtk.vtkLagrangianParticleTracker.STEP_CUR_CELL_VEL_DIR)
#     tracker.AdaptiveStepReintegrationOff()
#     tracker.Update()
    if (tracker.GetStepFactor() != 0.1):
        print('Incorrect StepFactor', file=sys.stderr)
        return
    if (tracker.GetStepFactorMin() != 0.1):
        print('Incorrect StepFactorMin', file=sys.stderr)
        return
    # Skipped other flags implemented in the cxx code

    sphereGlyph = vtk.vtkSphereSource()
    sphereGlyph.SetRadius(0.1)

#     points = vtk.vtkPoints()
#     points.InsertNextPoint(0, 0, 0)
#     points.InsertNextPoint(1, 1, 1)
#     points.InsertNextPoint(2, 2, 2)
#     polydata = vtk.vtkPolyData()
#     polydata.SetPoints(points)

#     glyph = vtk.vtkGlyph3D()
#     glyph.SetSourceConnection(sphereGlyph.GetOutputPort())
#     mbInter = vtk.vtkMultiBlockDataSet.SafeDownCast(tracker.GetOutput(1))
#     glyph.SetInputData(mbInter.GetBlock(1))

    mapper = vtk.vtkPolyDataMapper()
    mapper.SetInputData(vtk.vtkPolyData.SafeDownCast(tracker.GetOutput()))

    actor = vtk.vtkActor()
    actor.SetMapper(mapper)

    surfaceMapper = vtk.vtkPolyDataMapper()
    surfaceMapper.SetInputConnection(surfaceBounce.GetOutputPort())
    surfaceActor = vtk.vtkActor()
    surfaceActor.SetMapper(surfaceMapper)
    surfaceMapper2 = vtk.vtkPolyDataMapper()
    surfaceMapper2.SetInputConnection(surfacePass.GetOutputPort())
    surfaceActor2 = vtk.vtkActor()
    surfaceActor2.SetMapper(surfaceMapper2)

#     glyphMapper = vtk.vtkPolyDataMapper()
#     glyphMapper.SetInputConnection(glyph.GetOutputPort())
#     glyphActor = vtk.vtkActor()
#     glyphActor.SetMapper(glyphMapper)

    camera = vtk.vtkCamera()
    camera.SetFocalPoint(0, 0, -1)
    camera.SetViewUp(0, 0, 1)
    camera.SetPosition(0, -40, 0)

    renderer = vtk.vtkRenderer()
    renderer.SetActiveCamera(camera)
    renderWindow = vtk.vtkRenderWindow()
    renderWindow.AddRenderer(renderer)
    renderWindowInteractor = vtk.vtkRenderWindowInteractor()
    renderWindowInteractor.SetRenderWindow(renderWindow)
    renderer.AddActor(actor)
    renderer.AddActor(surfaceActor)
    renderer.AddActor(surfaceActor2)
#     renderer.AddActor(glyphActor)
    renderer.SetBackground(0.1, 0.5, 1)

    renderWindow.Render()
    renderWindowInteractor.Start()
    
    return

testLPT()

vtkLagrangianParticleTracker (0000022C1E9EC250)
  Debug: Off
  Modified Time: 1304
  Reference Count: 1
  Registered Events: (none)
  Executive: (none)
  ErrorCode: No error
  Information: 0000022C209BEEE0
  AbortExecute: Off
  Progress: 0
  Progress Text: (None)
  IntegrationModel: 0000000000000000
  Integrator: 0000000000000000
  CellLengthComputationMode: 0
  StepFactor: 1
  StepFactorMin: 0.5
  StepFactorMax: 1.5
  MaximumNumberOfSteps: 100
  MaximumIntegrationTime: -1
  AdaptiveStepReintegration: 0
  GenerateParticlePathsOutput: 1
  MinimumVelocityMagnitude: 0.001
  MinimumReductionFactor: 1.1
  ParticleCounter: 0
  IntegratedParticleCounter: 0




# Script to test few things. Can ignore

In [2]:
import vtk
seeds = vtk.vtkPointSource()
seeds.SetNumberOfPoints(10)
seeds.SetRadius(4)
seeds.Update()
seedPD = seeds.GetOutput()
seedData = seedPD.GetPointData()

partVel = vtk.vtkDoubleArray()
partVel.SetNumberOfComponents(3)
partVel.SetNumberOfTuples(seedPD.GetNumberOfPoints())
partVel.SetName('InitialVelocity')

partDens = vtk.vtkDoubleArray()
partDens.SetNumberOfComponents(1)
partDens.SetNumberOfTuples(seedPD.GetNumberOfPoints())
partDens.SetName('ParticleDensity')

partDiam = vtk.vtkDoubleArray()
partDiam.SetNumberOfComponents(1)
partDiam.SetNumberOfTuples(seedPD.GetNumberOfPoints())
partDiam.SetName('ParticleDiameter')

partVel.FillComponent(0, 2)
partVel.FillComponent(1, 5)
partVel.FillComponent(2, 1)
partDens.FillComponent(0, 1920)
partDiam.FillComponent(0, 0.1)

seedData.AddArray(partVel)
seedData.AddArray(partDens)
seedData.AddArray(partDiam)

point_data = seeds.GetOutput().GetPointData()


In [3]:
for i in range(point_data.GetNumberOfArrays()):
    array = point_data.GetArray(i)
    print(i, point_data.GetArrayName(i), '\t',
          'Points:', array.GetNumberOfTuples(),
          'Components:', array.GetNumberOfComponents(),
          'format:', array.GetArrayType()
          )

0 InitialVelocity 	 Points: 10 Components: 3 format: 2
1 ParticleDensity 	 Points: 10 Components: 1 format: 2
2 ParticleDiameter 	 Points: 10 Components: 1 format: 2


In [4]:
seeds.GetCenter()

(0.0, 0.0, 0.0)

In [5]:
seeds.GetRadius()

4.0